In [1]:
import requests
import json
import pandas as pd

In [2]:
csv_path = "show list.csv"

shows = pd.read_csv(csv_path)
shows = shows["Title"]
shows.head(20)

0                The Amazing Spider-Man
1                   The Incredible Hulk
2                            Spider-Man
3                                 Blade
4                Agents of S.H.I.E.L.D.
5                          Agent Carter
6                             Daredevil
7                         Jessica Jones
8                             Luke Cage
9                                Legion
10                            Iron Fist
11                        The Defenders
12                             Inhumans
13                           The Gifted
14                         The Punisher
15                             Runaways
16                       Cloak & Dagger
17                             Helstrom
18                          WandaVision
19    The Falcon and the Winter Soldier
Name: Title, dtype: object

In [3]:
base_url = "http://api.tvmaze.com/search/shows?q="

In [4]:

responses = []
show_info = []
show_name = []
show_id = []
tvrage = []
thetvdb = []
imdb = []



for show in shows: 
    mod_url = base_url + show
    
#     print(json.dumps(response_json, indent=4, sort_keys=True))

    
    try:
        show_data = requests.get(mod_url)
        response_json = show_data.json()
        show_name.append(response_json[0]['show']['name'])
        show_id.append(response_json[0]['show']['id'])
        tvrage.append(response_json[0]['show']['externals']['tvrage'])
        thetvdb.append(response_json[0]['show']['externals']['thetvdb'])
        imdb.append(response_json[0]['show']['externals']['imdb'])                       
        
    except:
        pass
                         
                         
                 

In [61]:
show_data_df = pd.DataFrame({"Show Name": show_name, "Show ID": show_id})
show_data_df.head()

,Show Name,Show ID
0,The Amazing Spider-Man,7623
1,The Incredible Hulk,2151
2,Spider-Man,1611
3,Blade,8331
4,Marvel's Agents of S.H.I.E.L.D.,31


In [6]:
# show_data_df['Show ID'] = show_data_df['Show ID'].astype(str)

In [7]:
cast_response = []
character_name = []
character_id = []
show_id_cast = []


for ids in show_id: 
    cast_url = (f"http://api.tvmaze.com/shows/{ids}?embed[]=episodes&embed[]=cast")
    
    
    try:
        cast_data = requests.get(cast_url).json()
        cast_response.append(cast_data)
        for i in range(0, 150): 
                character_name.append(cast_data['_embedded']['cast'][i]['character']['name'])
                character_id.append(cast_data['_embedded']['cast'][i]['character']['id'])
                show_id_cast.append(cast_data["id"])

    except:
        pass
                         
                   

In [8]:
character_data_df = pd.DataFrame({"Show ID": show_id_cast, "Character Name": character_name, "Character ID": character_id})
character_data_df

,Show ID,Character Name,Character ID
0,7623,Peter Parker/Spider-Man,196352
1,7623,Captain Barbera,365202
2,7623,J. Jonah Jameson,365203
3,7623,J. Jonah Jameson,365203
4,7623,Rita Conway,365204
...,...,...,...
441,33734,Rocket,725916
442,33734,Groot,725918
443,33734,Doctor Strange,725921
444,33734,She-Hulk,725928


In [10]:
episode_id = []
episode_season = []
episode_num = []
episode_air = []
episode_name = []
show_id_ep = []


for ids in show_id: 
    cast_url = (f"http://api.tvmaze.com/shows/{ids}?embed[]=episodes&embed[]=cast")
    
    
    try:
        cast_data = requests.get(cast_url).json()
        for i in range(0, 150): 
                episode_id.append(cast_data['_embedded']['episodes'][i]['id'])
                episode_season.append(cast_data['_embedded']['episodes'][i]['season'])
                episode_num.append(cast_data['_embedded']['episodes'][i]['number'])
                episode_air.append(cast_data['_embedded']['episodes'][i]['airstamp'])
                episode_name.append(cast_data['_embedded']['episodes'][i]['name'])
                show_id_ep.append(cast_data["id"])

    except:
        pass
                         
                   

In [46]:
episode_data_df = pd.DataFrame({"Show ID": show_id_ep, "Episode Name": episode_name, "Episode ID": episode_id, "Episode Name": episode_name, "Episode Season": episode_season, "Episode Number": episode_num, "Episode Air Date": episode_air})
episode_data_df = episode_data_df.groupby(["Show ID"])
episode_data_df.head()

,Show ID,Episode Name,Episode ID,Episode Season,Episode Number,Episode Air Date
0,7623,The Amazing Spider-Man,429649,1,1,1977-09-15T00:00:00+00:00
1,7623,The Deadly Dust: Part 1,429650,1,2,1978-04-06T01:00:00+00:00
2,7623,The Deadly Dust: Part 2,429651,1,3,1978-04-13T01:00:00+00:00
3,7623,The Curse of Rava,429652,1,4,1978-04-20T01:00:00+00:00
4,7623,Night of the Clones,429653,1,5,1978-04-27T01:00:00+00:00
...,...,...,...,...,...,...
2370,33734,One Big Mess,1371230,1,1,2017-10-13T16:00:00+00:00
2371,33734,"Uh Oh, It's Magic!",1371231,1,2,2017-10-13T16:00:00+00:00
2372,33734,That Drone Cat,1410751,1,3,2017-10-27T16:00:00+00:00
2373,33734,Rock and Roll,1410752,1,4,2017-10-27T16:00:00+00:00


In [64]:
episode_count_df = pd.DataFrame({"Total Episode Count": (episode_data_df["Episode ID"].count())})
episode_count_df

,Total Episode Count
Show ID,
31,136
336,18
439,26
472,76
671,52
...,...
41007,1
41748,9
41749,1


In [65]:
merged_df = pd.merge(character_data_df, episode_count_df, how='inner', on="Show ID")
merged_df

,Show ID,Character Name,Character ID,Total Episode Count
0,7623,Peter Parker/Spider-Man,196352,14
1,7623,Captain Barbera,365202,14
2,7623,J. Jonah Jameson,365203,14
3,7623,J. Jonah Jameson,365203,14
4,7623,Rita Conway,365204,14
...,...,...,...,...
419,33734,Rocket,725916,40
420,33734,Groot,725918,40
421,33734,Doctor Strange,725921,40
422,33734,She-Hulk,725928,40


In [66]:
name_merge_df = pd.merge(merged_df, show_data_df, how='inner', on="Show ID")
name_merge_df = name_merge_df[["Show ID", "Show Name", "Total Episode Count", "Character ID", "Character Name"]]
name_merge_df

,Show ID,Show Name,Total Episode Count,Character ID,Character Name
0,7623,The Amazing Spider-Man,14,196352,Peter Parker/Spider-Man
1,7623,The Amazing Spider-Man,14,365202,Captain Barbera
2,7623,The Amazing Spider-Man,14,365203,J. Jonah Jameson
3,7623,The Amazing Spider-Man,14,365203,J. Jonah Jameson
4,7623,The Amazing Spider-Man,14,365204,Rita Conway
...,...,...,...,...,...
461,33734,Marvel Super Hero Adventures,40,725916,Rocket
462,33734,Marvel Super Hero Adventures,40,725918,Groot
463,33734,Marvel Super Hero Adventures,40,725921,Doctor Strange
464,33734,Marvel Super Hero Adventures,40,725928,She-Hulk


In [ ]:
# Add network information???